In [279]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from tqdm import tqdm
#sbert imports  
# from sentence_transformers import SentenceTransformer, util
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# torch.set_printoptions(linewidth=220)

    Some kind soul already did a lot of [work](https://www.kaggle.com/azathoth42/myanimelist#anime_filtered.csv) on this topic, let's use it with thanks. I'll just use their cleaned dataframe.

In [280]:
df = pd.read_csv(r"C:\Users\kaush\Desktop\python_projects\anime_scraper\data\kaggle_anime_list\anime_cleaned.csv")
# cleaned_anime_df = pd.read_csv(r"C:\Users\kaush\Desktop\python_projects\anime_scraper\data\kaggle_anime_list\anime_cleaned.csv")
# filtered_anime_df = pd.read_csv(r"C:\Users\kaush\Desktop\python_projects\anime_scraper\data\kaggle_anime_list\anime_filtered.csv")
# base_anime_df = pd.read_csv(r"C:\Users\kaush\Desktop\python_projects\anime_scraper\data\kaggle_anime_list\AnimeList.csv")

In [281]:
# being modular just in case we decide to do the same on our scraped data or other dfs that the kaggle dataset provides
class CleanDataFrame():
    def __init__(self, df):
        self.df = df
        self.reduced_df = df[["anime_id", "title", "title_english", "rank"]]
        self.reduced_df = self.reduced_df.replace(np.nan, "Not available", regex=True)
        # dropping anime whose rank is not available
        self.reduced_df = self.reduced_df[self.reduced_df["rank"] != "Not available"]
        # converting rank to int
        self.reduced_df["rank"] = self.reduced_df["rank"].astype(int)
    
    def process_apostrophes(self, title_column, new_col_name):
        """ Dealing with apostrophes showing up as &#039; in titles """
        old_titles = self.reduced_df[title_column].tolist()
        new_titles = []
        for title in old_titles:
            if "&#039;" in title: #replacing &#039; with ', it is an artifact of the kaggle dataset
                title = title.replace("&#039;", "'")
            new_titles.append(title)
        self.reduced_df[new_col_name] = new_titles
         
    def populate_final_title_helper(self, row):
        """ Helper function to be as a lambda using df.apply() """
        final_title = row["title_english"] if row["title_english"] != "Not available" else row["title"]
        return final_title
           
    def get_final_title(self):
        """ Using english title as final title where available, else use title """
        self.reduced_df["final_title"] = self.reduced_df.apply(lambda row: self.populate_final_title_helper(row), axis=1)
    
    def remove_terms(self, columns, terms):
        """ Remove rows that contain certain blacklisted terms """
        for column in columns:
            self.reduced_df = self.reduced_df[~self.reduced_df[column].str.contains('|'.join(terms))]
            
    def clean(self):
        """ All cleaning functions run on the dataframe in one easy function """
        self.process_apostrophes("title", "title")
        self.process_apostrophes("title_english", "title_english")
        movie_terms = ['movie', 'Movie', 'MOVIE', 'film', 'Film', 'FILM']
        season_terms = ['season', 'Season', 'SEASON']
        self.remove_terms(["title", "title_english"], movie_terms)
        self.remove_terms(["title", "title_english"], season_terms)
              
    # def get_final_title_and_ranks(self):
    #     """ Return only final title """
    #     return self.reduced_df[["final_title", "rank"]]
    
    def save_top_n(self, n):
        """ Saving the top n anime by rank """
        pass
        

In [282]:
cdf = CleanDataFrame(df)
cdf.clean()
cdf.get_final_title()

In [283]:
trial_df = cdf.reduced_df.sort_values(by="rank")
df = trial_df

In [286]:
df.reset_index(inplace=True, drop=True)
df = df[["rank", "final_title"]]

In [290]:
df.to_excel(r"C:\Users\kaush\Desktop\python_projects\anime_scraper\data\top_anime_list\UsableAnimeList.xlsx", index=False)

In [266]:
# eng_titles = cdf.reduced_df.title_english.tolist()
# jap_titles = cdf.reduced_df.title.tolist()
# final_titles = cdf.reduced_df.final_title.tolist()

In [267]:
# model = SentenceTransformer('all-MiniLM-L6-v2')
# #Encode all sentences
# embeddings = model.encode(final_titles)

In [268]:
# cdf.reduced_df.sort_values(by="rank").title.tolist()

['Fullmetal Alchemist: Brotherhood',
 'Kimi no Na wa.',
 'Steins;Gate 0',
 'Steins;Gate',
 'Ginga Eiyuu Densetsu',
 'Hunter x Hunter (2011)',
 "Gintama': Enchousen",
 'Koe no Katachi',
 'Clannad: After Story',
 'Gintama',
 'Code Geass: Hangyaku no Lelouch R2',
 'Sen to Chihiro no Kamikakushi',
 'Made in Abyss',
 'Shigatsu wa Kimi no Uso',
 'Kizumonogatari III: Reiketsu-hen',
 'Shouwa Genroku Rakugo Shinjuu: Sukeroku Futatabi-hen',
 'Gintama.: Shirogane no Tamashii-hen',
 'Cowboy Bebop',
 'Hajime no Ippo',
 'Mononoke Hime',
 'Code Geass: Hangyaku no Lelouch',
 'Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen',
 'Mushishi Zoku Shou',
 'Ookami Kodomo no Ame to Yuki',
 'Great Teacher Onizuka',
 'Mushishi',
 'No Game No Life: Zero',
 'Tengen Toppa Gurren Lagann',
 'Hajime no Ippo: New Challenger',
 'Howl no Ugoku Shiro',
 'One Punch Man',
 'Suzumiya Haruhi no Shoushitsu',
 'Mushishi Zoku Shou: Suzu no Shizuku',
 'Kizumonogatari II: Nekketsu-hen',
 'Monster',
 'Death Note',
 'Saiki Kus

In [273]:
# len(final_titles)

5766

In [274]:
# def test_anime_presence(anime_title, anime_list=final_titles):
#     """ Testing if an anime substring is present in the list of anime """
#     results = []
#     for anime in anime_list:
#         if anime_title.lower() in anime.lower():
#             results.append(anime)
#     return results

In [275]:
# gintama_all = test_anime_presence("Gintama")
# fullmetal_all = test_anime_presence("Fullmetal")
# naru_all = test_anime_presence("Naruto")
# bleach_all = test_anime_presence("Bleach")
# onepiece_all = test_anime_presence("One Piece")

In [276]:
# gintama_all.extend(fullmetal_all)
# gintama_all.extend(naru_all)
# gintama_all.extend(bleach_all)
# gintama_all.extend(onepiece_all)

In [277]:
# def sim_df_producer(sentences, model='all-MiniLM-L6-v2'):
#     print(f"Length of the list of names given is: {len(sentences)}")
#     #define model to use
#     model = SentenceTransformer(model)
#     #Encode all sentences
#     embeddings = model.encode(sentences)
#     #Compute cosine similarity between all pairs
#     cos_sim = util.cos_sim(embeddings, embeddings)
#     # print(f"The cosine similarity matrix is: \n{cos_sim}")
#     print("Here's a more readable and prettier presentation:")
#     cos_sim_df = pd.DataFrame(cos_sim.numpy())
#     cos_sim_df.columns = sentences
#     cos_sim_df.index = sentences
#     return cos_sim_df.style.background_gradient(axis=0, cmap='YlOrRd')

In [278]:
# sim_df_producer(gintama_all)

Length of the list of names given is: 63
Here's a more readable and prettier presentation:


,Gintama.: Shirogane no Tamashii-hen,Gintama°: Umai-mono wa Atomawashi ni Suru to Yokodorisareru kara Yappari Saki ni Kue,Gintama: Yorinuki Gintama-san on Theater 2D,Gintama°: Aizome Kaori-hen,Gintama: Nanigoto mo Saiyo ga Kanjin nano de Tasho Senobisuru Kurai ga Choudoyoi,Gintama: Enchousen,Gintama,Gintama: Shiroyasha Koutan,Gintama.: Slip Arc,Gintama: Shinyaku Benizakura-hen,Gintama': Futon ni Haitte kara Buki Nokoshi ni Kizuite Neru ni Nerenai Toki mo Aru,Gintama: Dai Hanseikai,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood OVA Collection,Fullmetal Alchemist: Premium OVA Collection,Fullmetal Alchemist: The Sacred Star of Milos,Fullmetal Alchemist: The Sacred Star of Milos Specials,Fullmetal Alchemist: Brotherhood: 4-Koma Theater,Fullmetal Alchemist,Fullmetal Alchemist: Reflections,Naruto: Shippuden,Naruto: The Lost Story - Mission: Protect the Waterfall Village,NARUTO Spin-Off: Rock Lee & His Ninja Pals,Naruto x UT,Naruto,Boruto: Naruto Next Generations,Naruto: Find the Crimson Four-leaf Clover!,Chunin Exam on Fire! and Naruto vs. Konohamaru!,Naruto Narutimate Hero 3: Tsuini Gekitotsu! Jounin vs. Genin!! Musabetsu Dairansen taikai Kaisai!!,Naruto Shippuuden: Sunny Side Battle,Naruto: The Cross Roads,Bleach,Bleach Colorful!: Gotei Juusan Yatai Daisakusen!,Bleach: Memories in the Rain,Bleach: The Sealed Sword Frenzy,One Piece,One Piece 3D: Mugiwara Chase,One Piece: Mezase! Kaizoku Yakyuu Ou,One Piece: Straw Hat Theater,One Piece 3D2Y: Ace no shi wo Koete! Luffy Nakama Tono Chikai,One Piece: Defeat the Pirate Ganzack!,One Piece: Episode of Sabo - 3 Kyoudai no Kizuna Kiseki no Saikai to Uketsugareru Ishi,One Piece: Django's Dance Carnival,One Piece: Episode of Alabasta - Prologue,One Piece: Romance Dawn,One Piece: Long Ring Long Land-hen,One Piece: Yume no Soccer Ou!,One Piece: Episode of Merry - Mou Hitori no Nakama no Monogatari,"One Piece: Emergency Planning, A Perfect Strategy for the One Piece",One Piece Recap,One Piece: Umi no Heso no Daibouken-hen,One Piece 3D: Gekisou! Trap Coaster,One Piece: Romance Dawn Story,One Piece: Nenmatsu Tokubetsu Kikaku! Mugiwara no Luffy Oyabun Torimonochou,One Piece: Episode of East Blue - Luffy to 4-nin no Nakama no Daibouken,One Piece: Episode of Luffy - Hand Island no Bouken,One Piece: Episode of Nami - Koukaishi no Namida to Nakama no Kizuna,"One Piece Special: Open Upon the Great Sea! A Father's Huge, HUGE Dream!",Nissan Serena x One Piece 3D: Mugiwara Chase - Sennyuu!! Sauzando Sanii-gou,One Piece: Adventure of Nebulandia,One Piece: Heart of Gold,One Piece: Mamore! Saigo no Dai Butai,One Piece: Glorious Island
Gintama.: Shirogane no Tamashii-hen,1.000000,0.666483,0.586819,0.687925,0.698298,0.700870,0.760654,0.813072,0.645465,0.786103,0.657492,0.738838,0.169000,0.135262,0.097890,0.218746,0.206742,0.210504,0.143161,0.059102,0.392523,0.415421,0.312918,0.425534,0.472437,0.402357,0.395691,0.396821,0.488011,0.358302,0.409460,0.127596,0.451546,0.127646,0.179023,0.200778,0.311082,0.413094,0.148538,0.440349,0.287656,0.452729,0.204479,0.332376,0.260303,0.211488,0.170445,0.527596,0.095253,0.193785,0.494124,0.186790,0.236664,0.462347,0.458695,0.268509,0.504763,0.142651,0.301024,0.337203,0.186634,0.472394,0.194162
Gintama°: Umai-mono wa Atomawashi ni Suru to Yokodorisareru kara Yappari Saki ni Kue,0.666483,1.000000,0.465152,0.633430,0.651489,0.555277,0.594588,0.625239,0.524016,0.693130,0.684982,0.659658,0.162406,0.172953,0.156186,0.211469,0.207078,0.222715,0.175757,0.136083,0.313817,0.316617,0.225583,0.327497,0.328648,0.262602,0.267393,0.335575,0.420049,0.288735,0.312889,0.090943,0.424507,0.128173,0.112516,0.180594,0.257367,0.378028,0.141890,0.431906,0.142595,0.496933,0.092758,0.322053,0.251150,0.185313,0.170870,0.527694,0.103626,0.174088,0.472572,0.158927,0.247185,0.508265,0.457328,0.241918,0.528643,0.159452,0.271350,0.211122,0.179198,0.456903,0.206975
Gintama: Yorinuki Gintama-san on Theater 2D,0.586819,0.465152,1.000000,0.434615,0.491761,0.564638,0.631780,0.546176,0.571268,0.